# Cohere API and SciBERT for RAG - V5
This notebook uses a Cohere API for generating responses to text. A query input is required from the user. 
SciBERT is used for embeddings in a dense vector array both the text and the query. 
A DOI is supplied with the text as both an identifier and locator. 

- [x] set up venv
- [x] install transformers torch cohere in command line

### todo
- [ ] create script that compiles data/documents.txt with DOI || text for all documents
- [ ] reduce code by refactoring into modules
- [ ] store vectorized documents in a db
    - https://huggingface.co/learn/cookbook/rag_with_hugging_face_gemma_mongodb

### options
- Batch Processing:
    If large number of texts, process them in batches to avoid memory issues.
    Example: Use a loop or torch.utils.data.DataLoader.

- Change model size: smaller models require less processing

- fine tune model on corpus - i don't think this is an option

- look into pooling strategies
- concurrency? 

- Tokenizer
    - put cleaning process distincly prior to the tokenizer, using the default values as much as possible. 



In [1]:
# imports
import cohere
from cohere import Client
from transformers import AutoTokenizer, AutoModel
import numpy as np
from typing import List, Tuple, Dict
import os
from dotenv import load_dotenv
import json
import time # for timing functions
import sys

# load secret from local .env file
def get_key():
    #load secret .env file
    load_dotenv()

    #store credentials
    _key = os.getenv('COHERE_API_KEY')

    #verify if it worked
    if _key is not None:
        print("all is good, beautiful!")
        return _key
        

In [2]:

# initialize Cohere client with key from secrets
co = cohere.ClientV2(get_key())

# load SciBERT model and tokenizer
"""
Autotokenizer documentation can be found here: https://huggingface.co/docs/transformers/v4.50.0/en/model_doc/auto#transformers.AutoTokenizer

Model documentation can be found here: https://huggingface.co/allenai/scibert_scivocab_uncased
Citation for SciBERT:
@inproceedings{beltagy-etal-2019-scibert,
    title = "SciBERT: A Pretrained Language Model for Scientific Text",
    author = "Beltagy, Iz  and Lo, Kyle  and Cohan, Arman",
    booktitle = "EMNLP",
    year = "2019",
    publisher = "Association for Computational Linguistics",
    url = "https://www.aclweb.org/anthology/D19-1371"
}


"""
# Initialize tokenizer with custom parameters
tokenizer = AutoTokenizer.from_pretrained(
    "allenai/specter",
    max_len=512,
    #use_fast=True,  # Use the fast tokenizer
    do_lower_case=False,  # Preserve case
    add_prefix_space=False,  # No prefix space
    never_split=["[DOC]", "[REF]"],  # Tokens to never split
    #additional_special_tokens=["<doi>", "</doi>"],  # Add custom special tokens ***RE-EVALUATE*** (tuple or list of str or tokenizers.AddedToken, optional) — A tuple or a list of additional special tokens. Add them here to ensure they are skipped when decoding with skip_special_tokens is set to True. If they are not part of the vocabulary, they will be added at the end of the vocabulary.
    skip_special_tokens=False,
)

# this is the SciBERT model that is used to embed the text and query.
# other models: 'allenai-specter', 
#documentation here: https://huggingface.co/docs/transformers/model_doc/auto and https://github.com/allenai/specter
# load model directly
#tokenizer = AutoTokenizer.from_pretrained("allenai/specter")
model = AutoModel.from_pretrained("allenai/specter", torch_dtype="auto")

#model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased", torch_dtype="auto")
#may also want to consider using a sentence embedding model

###***** Specter2 use *********
"""
Should probably make a new notebook as it requires the use of adapters
Reference:
    https://huggingface.co/allenai/specter2
"""


#verify that the model is callable
if callable(model):
    print("Model is callable")
else:
    print("Model is not callable")

all is good, beautiful!
Model is callable


## V3: Dense retriever only
- set with parameters the same as BM25 pre-retriever version

calls a JSON file of documents

In [3]:
"""
Basic RAG using Cohere Command model
Dense retrieval of embedded query and pre-retrieved documents
Document source: data

Returns: responses based on query from input()
"""
# set top_k global value - keep this as constant for all evaluations
global top_k
top_k = 5

#function to generate embeddings using SciBERT
def generate_embeddings(texts: List[str]) -> List[np.ndarray]:
    """
    converts raw text to numerical representations using a pretrained model, in this case, SciBERT.
    Currently this is applied to both the document text and the query. 
    May want a different version or decorator for the query as they are generally much shorter and more sparse.

    Input: text from tokenizer step above as a list of strings
    Output: np.array
    """
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        max_length=512,
        padding="max_length",
        truncation=True,
        return_attention_mask=True)
    # this passes the tokenized inputs through the model
    outputs = model(**inputs)
    #this uses mean pooling - may want to investigate other methods
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

#read documents as .txt files in data director
def read_documents_with_doi(directory_path: str) -> List[Dict[str, str]]:
    """
    Reads documents and their DOIs from individual files in a directory.

    Args:
        directory_path (str): Path to the directory containing the document files.

    Returns:
        List[Dict[str, str]]: A list of dictionaries, each containing 'doi' and 'text' keys.
    """
    documents_with_doi = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, "r", encoding="utf-8") as file:
                lines = file.readlines()
                if len(lines) >= 1:
                    doi = lines[0].strip().replace("DOI: ", "")
                    text = "".join(lines[1:]).strip()
                    documents_with_doi.append({"doi": doi, "text": text})
    return documents_with_doi


# Function to update chat history
def update_chat_history(query, retrieved_docs, response)->None:
    global chat_history
    try:
        chat_history.append({
            "query": query,
            "retrieved_docs": [doc["text"] for doc in retrieved_docs],  # Store only the text of retrieved documents
            "response": response
        })
    except:
        chat_history.append({
            "query": query,
            "retrieved_docs": None,
            "response": response
        })


#function to incorporate history into the next query
def get_context_with_history(query) -> str:
    global chat_history # also declare here since chat_history is being modified
    if not chat_history:
        return None
    else:
        for entry in chat_history:
            history_str = "\n".join([
                f"User: {entry['query']}\n"
                f"Context: {'; '.join(entry['retrieved_docs'])}\n"
                f"Response: {entry['response']}"])
     
    return f"Chat History:\n{history_str}\nCurrent Query: {query}"

#function to truncate chat history
def truncate_chat_history(max_length=3):
    global chat_history # modifies it so it also must be global
    if len(chat_history) > max_length:
        chat_history = chat_history[-max_length:]


def retrieve_documents(query: str, threshold:float, documents:List) -> List[Dict[str, str]]:
    """
    Dense retriever: retrieves documents from the embedded documents and performs cosine similarity for similarities score

    Args:
        query: this is the query passed
        threshold: value for similarity cutoff
    Returns:
        List of dictionaries containing strings as key/value pairs
    """
    query_embedding = generate_embeddings([query])[0]
    document_embeddings = generate_embeddings(documents)
    #cosine similarity
    similarities = [
        np.dot(query_embedding, doc_emb) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb))
        for doc_emb in document_embeddings
    ]
    # Filter documents based on the threshold
    filtered_indices = [i for i, sim in enumerate(similarities) if sim >= threshold]
    #sorts on similarity score then reverses order, slices only to top_k
    top_indices = np.argsort(similarities)[::-1][:top_k]
    try:
        return [documents_with_doi[i] for i in top_indices]
    except:
        return None

def retrieve_documents_2(query:str, threshold:float, documents:List,documents_with_doi:List) -> List[Dict[str,str]]:
    """
    Dense retriever: retrieves documents from the embedded documents and performs cosine similarity for similarities score

    Args:
        query: this is the query passed
        threshold: value for similarity cutoff
    Returns:
        List of dictionaries containing strings as key/value pairs
    """
    query_embedding = generate_embeddings([query])[0]
    document_embeddings = generate_embeddings(documents)
    #cosine similarity
    similarities = [
        np.dot(query_embedding, doc_emb) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb))
        for doc_emb in document_embeddings
    ]
    #test_tuple = (i,float(sim) for i,sim in enumerate(similarities) if sim >= threshold)
    test_tuple = []
    for i,sim in enumerate(similarities):
        if sim >= threshold:
            a = (i,sim)
            test_tuple.append(a)
    toppy_top = sorted(test_tuple,key=lambda score: score[1], reverse=True)
    filtered_list = toppy_top[:5]
    top_indices = [idx for idx,score in filtered_list]
    doc_list = [documents_with_doi[i].get('doi') for i in top_indices]
    score_list = [float(score) for idx,score in filtered_list]

    score_tuple = list(zip(score_list,doc_list))

    try:
        return [documents_with_doi[i] for i in top_indices],score_tuple
    except:
        return None

#RAG pipeline function
def rag_pipeline(query):


    # Main loop for user interaction 
    # changed to run with no history for the purposes of the test
    global chat_history
    chat_history = []#initialize chat history
    
    #incorporate chat history
    full_context = get_context_with_history(query)
    # let user know you are generating...
    print("Retrieving documents and generating response...")

    #retrieve documents
    # ***** SET THRESHOLD LEVEL HERE!********
    #retrieved_docs = retrieve_documents(query,0.80)
    retrieved_docs,filtered_list_with_scores = retrieve_documents_2(query,0.1,documents,documents_with_doi)


    #prepare context for Cohere's Command model
    instruction = """You are an academic research assistant.
                    If there are 0 documents in context, then state that you can not provide an answer.
                    If there is context, your response should be structured in a paragraph of less than 250 words in the following order: 
                    Summary answer of once sentence.
                    DOI: {insert DOI here} - Summary of supporting document 1
                    DOI: {insert DOI here} - Summary of supporting document 2
                    Concluding statement. 
                    """  
    # add full_context if exists, else just context. 
    if retrieved_docs:    
        if full_context:
            context = "\n".join([f"DOI: {doc['doi']}, Text: {doc['text']}" for doc in retrieved_docs]).join(full_context)
        else:
            context = "\n".join([f"DOI: {doc['doi']}, Text: {doc['text']}" for doc in retrieved_docs])
    else:
        context = f"There are 0 documents related to your query."
    prompt = f"Instruction: {instruction}\nQuery: {query}\nContext: {context}\n"
    
    
    # Generate response - updated to V2 - see docs: https://docs.cohere.com/reference/chat

    #V2
    response = co.chat(
        model="command-a-03-2025",
        #model="command-r7b-12-2024", #https://docs.cohere.com/docs/command-r7b
        messages=[{"role": "user", "content": prompt}],
        ).message.content[0].text
    # Update chat history
    update_chat_history(query, retrieved_docs, response)
    
    # Truncate history if necessary
    truncate_chat_history()

    # Print the response
    print("\nGenerated Response:")
    print(response)
    print(f"------\nSource documents: ")#used for debugging
    try:
        for doc in retrieved_docs:
            doi_with_retriever = f"https://doi.org/{doc['doi']}"
            print(f"DOI: {doi_with_retriever}, {doc['text'].split("\n",1)[0]}")
        return response, retrieved_docs, filtered_list_with_scores
    except:
        print(f"No documents found")
        return response, retrieved_docs, filtered_list_with_scores

#***** Begin chat session *****

# Path to the file containing documents and DOIs
#directory_path = "/Users/poppyriddle/Documents/PhD/Research_proposal/Part_3/part_3_cohere/data"
#directory_path = "/Users/poppyriddle/Documents/PhD/Research_proposal/Part_3/part_3_cohere/data_jats"
#directory_path = "/Users/poppyriddle/Documents/PhD/Research_proposal/Part_3/part_3_cohere/data_multi_lang"

# Read documents and DOIs from the file
#documents_with_doi = read_documents_with_doi(directory_path)
#documents = [doc["text"] for doc in documents_with_doi]
#print(f"Length of documents: {len(documents)}")
#print(f"Length of corpus: {len(documents_with_doi)}")

#query = input("What is your query (or type 'exit' to quit): ")

#rag_pipeline(query)



# Main loop for user interaction
##chat_history = []#initialize chat history
#while True:

    #uery = input("What is your query (or type 'exit' to quit): ")
    #if query.lower() == "exit":
    #    break
    #rag_pipeline(query)

    #print(f"time to query loop: {time_query:.2f} seconds")
    #print(f"to to retrieve: {retrieve_time:.2f} seconds")
    #print(f"time to generate: {generate_time:.2f} seconds")


# Analysis
## Test One: 
- [✅] compute precision, recall, and F1-Scores. 
- [✅] added accuracy score
- [ ] compare text from each source, embedded, and them similarity scores based on embeddings.
    - [ ] token based SciBERT embedding
    - [ ] sentence-based SentenceBERT embedding
### optional analysis
Need to learn more about attention weights and their analysis
- [✅] heatmap of attention weights for two given inputs


## Precision, recall, F1 score
### references
- https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html
- https://vitalflux.com/accuracy-precision-recall-f1-score-python-example/

In [5]:
from typing import List, Dict, Set
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, balanced_accuracy_score
import openpyxl
import pandas as pd
import numpy as np
from colorama import Fore, Back, Style

In [16]:
#initial dataframe to capture results from each query and results
#ONLY DO THIS AT THE BEGINNING OF THE ANALYSIS PROCEDURE, OTHERWISE, IT WILL ERASE THE PREVIOUS RESULTS!!

results_df = pd.DataFrame(columns=['Query','Precision','Recall','F1-Score','Accuracy', 'Balanced accuracy', 'Faithfulness score', 'Documents score', 'Response'])
results_df

,Query,Precision,Recall,F1-Score,Accuracy,Balanced accuracy,Faithfulness score,Documents score,Response


In [11]:

"""
evaluate_retrieval() function takes 4 inputs and generates a dictionary of results to be 
printed out by the function print_results()

print_results() function takes the 4 inputs and passes them to evaluate_retrieval(). 
print_results is the only one call in the test_loop() function and calls evaluate_retrieval() for 
each run of the pipeline function. 

"""

def evaluate_retrieval(retrieved_dois, ground_truth, response, query:str,tuple_list_with_scores)->Dict:
    corpus_doi_list = []
    #corpus_list is a global variable in rag_pipeline()
    for each in range(len(documents_with_doi)):
        a = documents_with_doi[each].get('doi',"")
        corpus_doi_list.append(a)
    print(len(corpus_doi_list))

    def compare_lists(list1, list2, list3):
        for val in list1:
            if val in list2:
                list3.append(1)
            else:
                list3.append(0)

    #set y_true so that len(y_true)==len(corpus_doi_list)
    y_true = []
    compare_lists(corpus_doi_list,ground_truth,y_true)
    y_true = np.array(y_true)
    y_pred = []
    compare_lists(corpus_doi_list,retrieved_dois,y_pred)


    # calculate metrics - could also use sklearn.metrics functions such as precision_score, but this is easier to read
    precision = precision_score(y_true, y_pred, average="micro")
    recall = recall_score(y_true, y_pred,average="micro")
    f1 = f1_score(y_true, y_pred, average="micro")
    accuracy = accuracy_score(y_true, y_pred, normalize=True)
    balanced_accuracy = balanced_accuracy_score(y_true, y_pred)

    faithfulness_score = 0
    for each in retrieved_dois:
        if each in response:
            faithfulness_score+=1
        else:
            faithfulness_score+=0

        
    return {
        'Query':f"{query}",
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "Accuracy":accuracy,
        "Balanced accuracy":balanced_accuracy,
        "Faithfulness score":faithfulness_score,
        "Documents score":str(tuple_list_with_scores),
        "Response":response
    }

def print_results(retrieved_dois, ground_truth, response, query:str, tuple_list_with_scores)->Dict:
    """
    Prints a nicely ordered set of results from evalaute_retrieval()
    """

    results = evaluate_retrieval(retrieved_dois, ground_truth, response, query, tuple_list_with_scores)
    print(f"For query: {results['Query']}:")
    print(f"Precision: {results['Precision']:.3f}")
    print(f"Recall: {results['Recall']:.3f}")
    print(f"F1-Score: {results['F1-Score']:.3f}")
    print(f"Accuracy: {results['Accuracy']:.3f}")
    print(f"Balanced accuracy: {results['Balanced accuracy']:.3f}")
    print(f"Faithfulness score: {results['Faithfulness score']}")
    print(f"Documents score: {results['Documents score']}")
    return results

#print_results()


## Run the test

- [✅] load golden_set.xlsx
- [✅] iterate through the list 5 times
- [✅] add sleep that prevents API overusage

In [15]:
golden_set_df = pd.read_excel("golden_set.xlsx")
#golden_set_df_test = golden_set_df.head(3)
golden_set_df


,query,ground_truth,n_ground_truth,expected_response,Refernces:,Response\nBM25,Response\nDense
0,which studies examined the abstract in metadata?,"[""10.1002/leap.1411"",""10.1007/s11192-020-03632...",5,Lexical content changes in abstracts during th...,DOI: 10.1002/leap.1411\nInvestigates lexical c...,NaN,NaN
1,which studies examined citations?,"[""10.1007/s11192-022-04367-w"",""10.1371/journal...",5,Identifying and correcting invalid citations d...,DOI: 10.1007/s11192-022-04367-w\nFocuses on id...,NaN,NaN
2,Tell me about OpenAlex.,"[""10.3145/epi.2023.mar.09"",""10.1590/SciELOPrep...",7,OpenAlex is presented as a promising open-sour...,DOI: 10.3145/epi.2023.mar.09\nCompares OpenAle...,NaN,NaN
3,Tell me about Crossref.,"[""10.1162/qss_a_00212"",""10.31274/b8136f97.ccc3...",9,Crossref is portrayed as a major source of sch...,DOI: 10.1162/qss_a_00212 - Examines Crossref’s...,NaN,NaN
4,Which papers evaluate the linguistic coverage ...,"[""10.1007/s11192-015-1765-5"",""10.48550/arXiv.2...",5,Biases in Traditional Databases: WoS and Scopu...,DOI: 10.1007/s11192-015-1765-5 - Compares the ...,NaN,NaN
5,"Which papers address funding metadata, its ava...","[""10.1162/qss_a_00210"",""10.1162/qss_a_00212"",""...",5,Assessing Availability: Highlighting gaps in f...,DOI: 10.1162/qss_a_00210 - Examines the availa...,NaN,NaN
6,Which papers discuss the use of Retrieval-Augm...,"[""10.1007/978-3-031-88708-6_3"",""10.1609/aaai.v...",5,Evaluation and Benchmarking: Diagnosing RAG’s ...,DOI: 10.1007/978-3-031-88708-6_3 \nInvestigate...,NaN,NaN
7,What is Crossref’s role in the scholarly resea...,"[""10.1162/qss_a_00022"",""10.1162/qss_a_00210"",""...",5,Crossref plays a central role in the scholarly...,DOI: 10.1162/qss_a_00022\n Describes Crossref ...,NaN,NaN
8,What are the key features and limitations of O...,"[""10.3145/epi.2023.mar.09"",""10.1590/SciELOPrep...",5,OpenAlex is highlighted as a promising open-so...,DOI: 10.3145/epi.2023.mar.09\nKey Features: Pr...,NaN,NaN
9,What are the strengths and weaknesses of Web o...,"[""10.1007/s11192-015-1765-5"",""10.1162/qss_a_00...",5,Web of Science (WoS) is recognized for its str...,DOI: 10.1007/s11192-015-1765-5\nStrengths: Com...,NaN,NaN


In [17]:


# Path to the file containing documents and DOIs
#directory_path = "/Users/poppyriddle/Documents/PhD/Research_proposal/Part_3/part_3_cohere/data"
#directory_path = "/Users/poppyriddle/Documents/PhD/Research_proposal/Part_3/part_3_cohere/data_jats"
directory_path = "/Users/poppyriddle/Documents/PhD/Research_proposal/Part_3/part_3_cohere/data_multi_lang"

# Read documents and DOIs from the file
documents_with_doi = read_documents_with_doi(directory_path)
documents = [doc["text"] for doc in documents_with_doi]
print(f"Length of documents: {len(documents)}")
print(f"Length of corpus: {len(documents_with_doi)}")

# define test loop
def test_loop(query:str,ground_truth:List[str]):
    #run the test from here
    # set top_k global value - keep this as constant for all evaluations
    global top_k
    top_k = 5
    #***** Begin chat session *****
    response,retrieved_docs,tuple_list_with_scores = rag_pipeline(query)

    # Extract DOIs from retrieved documents
    retrieved_dois = [doc.get('doi', "") for doc in retrieved_docs]
    print("Retrieved DOIs:", retrieved_dois)

    new_result = print_results(retrieved_dois, ground_truth, response, query, tuple_list_with_scores)
    print(Fore.LIGHTRED_EX + f"{tuple_list_with_scores}")
    results_df.loc[len(results_df)] = new_result

    #save the queries and responses to separate dataframe to be manually annontated
    answer_relevance_df = results_df[['Query','Response']].copy(deep=True)

    # save out answer_relevance_df
    filename="analysis/dense_answer_relevance_results.xlsx"
    answer_relevance_df.to_excel(filename)

    filename = "analysis/dense_analysis_results.xlsx"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    results_df.to_excel(filename)

    #time sleep to avoid exceeding API limit
    sleepy_time = 45
    print(Fore.LIGHTMAGENTA_EX + f"sleeping for {sleepy_time} seconds...")
    time.sleep(sleepy_time)
    print(Fore.LIGHTBLUE_EX + f"Next one....")
    return results_df

#golden_set_df_test['Response\nDense'] = golden_set_df_test.apply(lambda x: test_loop(x.query,x.ground_truth), axis=1)
golden_set_df_query = golden_set_df['query'].to_list()
golden_set_df_ground_truth = golden_set_df['ground_truth'].to_list()

loop_length = 5
while loop_length:
    for i in range(len(golden_set_df_query)):
        test_loop(golden_set_df_query[i],golden_set_df_ground_truth[i])
        print(Fore.LIGHTCYAN_EX + f"Working on row: {i}")
    loop_length = loop_length-1

print(Fore.LIGHTMAGENTA_EX + f"!!!!! All Done!!!!!")



Length of documents: 45
Length of corpus: 45
Retrieving documents and generating response...

Generated Response:
**Summary answer:** Several studies have examined the availability and quality of abstracts in metadata across various bibliographic databases, including Crossref, OpenAlex, Scopus, Web of Science, and others.  

DOI: 10.31222/osf.io/smxe5 - This study evaluates the open availability of six metadata elements in Crossref, including abstracts, and finds that while availability has improved over time for journal articles, many publishers still need to enhance full openness of bibliographic metadata.  
DOI: 10.48550/arXiv.2406.15154 - The research compares publication and document types across OpenAlex, Scopus, Web of Science, Semantic Scholar, and PubMed, highlighting differences in how abstracts and other metadata are classified and made available across these platforms.  
DOI: 10.5860/crl.86.1.101 - This study identifies metadata quality issues, including those related to ab

In [1]:
documents_with_doi

NameError: name 'documents_with_doi' is not defined

# Now check your results in the analysis folder


## compare text embeddings for each source
This section is used to investigate the impact of face markup or other formatting requirements in the abstract text on the embedding model.
Currently this only uses cosine similarity. 
<br>
#### similarity:
- [✅] cosine similarity
<br>
#### Embeddings:
- [✅] SciBERT
- [✅] SentenceBERT
<br>
#### Analysis
- [ ] compare scores using dataframe
- [ ] visualize results
### References
- https://stackoverflow.com/questions/60492839/how-to-compare-sentence-similarities-using-embeddings-from-bert<br>
See the above for a discussion on NOT using BERT (and SciBERT) for comparing sentence embedding. I should be using SentenceBERT for sentence similarity.<br>
- Sentence Transformers: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html
<br>
- another approach: https://medium.com/@ahmedmellit/text-similarity-implementation-using-bert-embedding-in-python-1efdb5194e65
- sklearn metrics for other scoring methods than cosine similarity: https://scikit-learn.org/stable/api/sklearn.metrics.html

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

#load SciBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')

def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

#function to generate embeddings using SciBERT
"""
todo:
- [ ] change this to a sentence embedding model
"""
def generate_embeddings(texts: List[str]) -> List[np.ndarray]:
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        max_length=512,
        padding="longest",#please select one of ['longest', 'max_length', 'do_not_pad']
        #padding=False,#padding has an effect on similarity
        truncation=True
    )
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()

    return embeddings

#original text and text with errors
original_text = " Abstract Objectives Precise literature recommendation and summarization are crucial for biomedical professionals. While the latest iteration of generative pretrained transformer (GPT) incorporates 2 distinct modes—real-time search and pretrained model utilization—it encounters challenges in dealing with these tasks. Specifically, the real-time search can pinpoint some relevant articles but occasionally provides fabricated papers, whereas the pretrained model excels in generating well-structured summaries but struggles to cite specific sources. In response, this study introduces RefAI, an innovative retrieval-augmented generative tool designed to synergize the strengths of large language models (LLMs) while overcoming their limitations. Materials and Methods RefAI utilized PubMed for systematic literature retrieval, employed a novel multivariable algorithm for article recommendation, and leveraged GPT-4 turbo for summarization. Ten queries under 2 prevalent topics (“cancer immunotherapy and target therapy” and “LLMs in medicine”) were chosen as use cases and 3 established counterparts (ChatGPT-4, ScholarAI, and Gemini) as our baselines. The evaluation was conducted by 10 domain experts through standard statistical analyses for performance comparison. The overall performance of RefAI surpassed that of the baselines across 5 evaluated dimensions—relevance and quality for literature recommendation, accuracy, comprehensiveness, and reference integration for summarization, with the majority exhibiting statistically significant improvements (P-values<.05). Discussion RefAI demonstrated substantial improvements in literature recommendation and summarization over existing tools, addressing issues like fabricated papers, metadata inaccuracies, restricted recommendations, and poor reference integration. Conclusion By augmenting LLM with external resources and a novel ranking algorithm, RefAI is uniquely capable of recommending high-quality literature and generating well-structured summaries, holding the potential to meet the critical needs of biomedical professionals in navigating and synthesizing vast amounts of scientific literature."
typo_text = "<jats:title>Abstract</jats:title>\n               <jats:sec>\n                  <jats:title>Objectives</jats:title>\n                  <jats:p>Precise literature recommendation and summarization are crucial for biomedical professionals. While the latest iteration of generative pretrained transformer (GPT) incorporates 2 distinct modes—real-time search and pretrained model utilization—it encounters challenges in dealing with these tasks. Specifically, the real-time search can pinpoint some relevant articles but occasionally provides fabricated papers, whereas the pretrained model excels in generating well-structured summaries but struggles to cite specific sources. In response, this study introduces RefAI, an innovative retrieval-augmented generative tool designed to synergize the strengths of large language models (LLMs) while overcoming their limitations.</jats:p>\n               </jats:sec>\n               <jats:sec>\n                  <jats:title>Materials and Methods</jats:title>\n                  <jats:p>RefAI utilized PubMed for systematic literature retrieval, employed a novel multivariable algorithm for article recommendation, and leveraged GPT-4 turbo for summarization. Ten queries under 2 prevalent topics (“cancer immunotherapy and target therapy” and “LLMs in medicine”) were chosen as use cases and 3 established counterparts (ChatGPT-4, ScholarAI, and Gemini) as our baselines. The evaluation was conducted by 10 domain experts through standard statistical analyses for performance comparison.</jats:p>\n               </jats:sec>\n               <jats:sec>\n                  <jats:title>Results</jats:title>\n                  <jats:p>The overall performance of RefAI surpassed that of the baselines across 5 evaluated dimensions—relevance and quality for literature recommendation, accuracy, comprehensiveness, and reference integration for summarization, with the majority exhibiting statistically significant improvements (P-values &amp;lt;.05).</jats:p>\n               </jats:sec>\n               <jats:sec>\n                  <jats:title>Discussion</jats:title>\n                  <jats:p>RefAI demonstrated substantial improvements in literature recommendation and summarization over existing tools, addressing issues like fabricated papers, metadata inaccuracies, restricted recommendations, and poor reference integration.</jats:p>\n               </jats:sec>\n               <jats:sec>\n                  <jats:title>Conclusion</jats:title>\n                  <jats:p>By augmenting LLM with external resources and a novel ranking algorithm, RefAI is uniquely capable of recommending high-quality literature and generating well-structured summaries, holding the potential to meet the critical needs of biomedical professionals in navigating and synthesizing vast amounts of scientific literature.</jats:p>\n               </jats:sec>"

#run embeddings
original_embedding = generate_embeddings(original_text)
typo_embedding = generate_embeddings(typo_text)
print(type(typo_embedding))

#calculate cosine similarity
similarity = cosine_similarity(original_embedding, typo_embedding)
print(f"Cosine similarity: {similarity[0][0]:.4f}")

In [ ]:
#sentence transformer verison
#reference: https://medium.com/@ahmedmellit/text-similarity-implementation-using-bert-embedding-in-python-1efdb5194e65

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

#puts text from above into a list
sentences:list = [original_text,typo_text]

#initializing the Sentence Transformer model using BERT with mean-tokens pooling - source see above
sentence_model = SentenceTransformer('bert-base-nli-mean-tokens') # this resets the model variable! changed to sentence_model variable name

#encoding the sentences
sentence_embeddings = sentence_model.encode(sentences)

#result will be a list of similarity scores between two texts
similarity_scores = cosine_similarity([sentence_embeddings[0]], sentence_embeddings[1:])

print(f"Cosine similarity scores using sentence embedding model: {similarity_scores[0][0]:.4f}")



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_heatmap(embedding1, embedding2, title):
    #calculate the difference between the two embeddings
    diff = embedding1 - embedding2
    #reshape the difference to a 2D array for the heatmap
    diff_2d = diff.reshape(1, -1)
    #create a heatmap
    plt.figure(figsize=(12, 2))
    sns.heatmap(diff, cmap='coolwarm', annot=False, cbar=True,vmin=-1,vmax=1)
    plt.title(title)
    plt.show()

#plot heatmap
#plot_heatmap(original_embedding, typo_embedding, "diff between embeddings")
plot_heatmap(sentence_embeddings[0],sentence_embeddings[1:],"Sentence Embeddings")

### with attention weights
from https://github.com/clarkkev/attention-analysis
- https://stackoverflow.com/questions/75772288/how-to-read-a-bert-attention-weight-matrix for explanation on queries and keys
- https://theaisummer.com/self-attention/#:%7E:text=Self%2Dattention%20is%20not%20symmetric!&text=The%20arrows%20that%20correspond%20to,Q%E2%80%8B%3DWK%E2%80%8B. explanation on self-attention
- heatmaps to analyze attention weights: https://apxml.com/courses/foundations-transformers-architecture/chapter-7-implementation-details-optimization/practice-analyzing-attention-weights
- excellent source: https://apxml.com/courses/how-to-build-a-large-language-model/chapter-23-analyzing-model-behavior/attention-map-visualization


In [ ]:
# same as above 
import torch
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
import seaborn as sns

# Load SciBERT tokenizer and model - same as above - technically don't need to relaod these unless changing
# try sentence based model?
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')


# Tokenize the sentences
inputs1 = tokenizer(original_text, return_tensors="pt", padding='max_length', truncation=True, max_length=20)#limit tokens so that we can actually see something
inputs2 = tokenizer(typo_text, return_tensors="pt", padding='max_length', truncation=True, max_length=20)#

# Get the attention weights: the `output_attentions=True` parameter is used to get the attention weights from the model
with torch.no_grad():
    outputs1 = model(**inputs1, output_attentions=True)
    outputs2 = model(**inputs2, output_attentions=True)

# Extract the attention weights for the last layer
#.squeeze() https://numpy.org/doc/stable/reference/generated/numpy.squeeze.html
attention_weights1 = outputs1.attentions[-1].squeeze(0)  # Shape: (num_heads, seq_len, seq_len)
attention_weights2 = outputs2.attentions[-1].squeeze(0)  # Shape: (num_heads, seq_len, seq_len)

# Average the attention weights across all heads, 
#see last reference to visualize attention for each head
attention_weights1 = attention_weights1.mean(dim=0)  # Shape: (seq_len, seq_len)
attention_weights2 = attention_weights2.mean(dim=0)  # Shape: (seq_len, seq_len)

# Get the tokens for the sentences
tokens1 = tokenizer.convert_ids_to_tokens(inputs1["input_ids"].squeeze(0))
tokens2 = tokenizer.convert_ids_to_tokens(inputs2["input_ids"].squeeze(0))

# Plot the attention heatmap for the first sentence
plt.figure(figsize=(10, 8))
sns.heatmap(attention_weights1, xticklabels=tokens1, yticklabels=tokens1, cmap='viridis', annot=False, cbar=True)
plt.title("Attention Weights for original_text")
plt.show()

# Plot the attention heatmap for the second sentence
plt.figure(figsize=(10, 8))
sns.heatmap(attention_weights2, xticklabels=tokens2, yticklabels=tokens2, cmap='viridis', annot=False, cbar=True)
plt.title("Attention Weights for typo_text")
plt.show()

# Calculate the difference in attention weights
diff_attention_weights = (attention_weights1 - attention_weights2)

# Plot the **difference** in attention weights
plt.figure(figsize=(10, 8))
sns.heatmap(diff_attention_weights, xticklabels=tokens1, yticklabels=tokens1, cmap='coolwarm', annot=False, cbar=True, vmin=-1, vmax=1)
plt.title("Difference in Attention Weights")
plt.show()
